In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import Image
sns.set()

In [ ]:
pip install matplotlib_venn --upgrade --user

https://www.glerl.noaa.gov/metdata/mil/archive/ 

In [11]:
pip install pandas --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install numexpr --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install seaborn --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install scipy --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [15]:
df = pd.read_csv("mkecallswheader.csv")

In [19]:
df.head(10)

,id,time,location,district,nature,status,inserted_at,updated_at,point,call_id
0,2093116,2019-05-21 15:19:03,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,2019-05-21 20:51:09,2019-05-21 20:51:09,0101000020E6100000FC7C94111793454061D971683600...,191411633
1,2093127,2019-05-21 15:24:30,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,2019-05-21 20:57:11,2019-05-21 20:57:11,0101000020E6100000D2AB014A4386454067C416CCA9FC...,191411672
2,2093141,2019-05-21 15:25:46,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,2019-05-21 21:00:12,2019-05-21 21:00:12,0101000020E610000053FFC5D8AE8B45402CAE3B270700...,191411674
3,2093805,2019-05-21 20:46:28,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,2019-05-22 02:22:32,2019-05-22 02:22:32,0101000020E610000078ABF8D04F804540633ABE0779FA...,191412545
4,2093816,2019-05-21 20:50:03,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,2019-05-22 02:25:33,2019-05-22 02:25:33,0101000020E6100000E8323509DE834540C3D7D7BAD4FB...,191412465
5,2093829,2019-05-21 21:02:37,"E WRIGHT ST / N WEIL ST,MKE",5,PARK AND WALK,Service in Progress,2019-05-22 02:37:36,2019-05-22 02:37:36,0101000020E6100000DEF1DC312B88454059D878558CF9...,191412584
6,2093872,2019-05-21 20:50:47,"9010 N 95TH ST,MKE",4,WELFARE CITIZEN,Advised,2019-05-22 02:52:43,2019-05-22 02:52:43,0101000020E6100000357D76C07597454080B4FF01D601...,191412544
7,2093887,2019-05-21 21:25:33,"983 W ARTHUR AV,MKE",2,BATTERY DV,Service in Progress,2019-05-22 03:01:48,2019-05-22 03:01:48,0101000020E6100000BC033C69E17F454041ABDDC02EFB...,191412632
8,2093918,2019-05-21 21:36:05,"4115 N 56TH ST,MKE",7,RETURN STATION,Assignment Completed,2019-05-22 03:16:51,2019-05-22 03:16:51,0101000020E610000014483FD0C08B45404EB747CAF1FE...,191412656
9,2093929,2019-05-21 21:45:53,"7806 W HAMPTON AV,MKE",7,TRAFFIC STOP,Advised,2019-05-22 03:22:52,2019-05-22 03:22:52,0101000020E6100000F6F0C05B7B8D45404F34B4A69E00...,191412676


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id           int64 
 1   time         object
 2   location     object
 3   district     object
 4   nature       object
 5   status       object
 6   inserted_at  object
 7   updated_at   object
 8   point        object
 9   call_id      int64 
dtypes: int64(2), object(8)
memory usage: 307.3+ MB


https://mpd.digitalpublicworks.com/?start=2019-01-05T00:00:00-06:00&end=2019-01-05T23:59:59.999999-06:00